# V1 找出結尾與開頭 4. 

In [8]:
from ckipnlp.pipeline import CkipPipeline
import regex as re
import pandas as pd
from tqdm import trange
import ast

## 載檔

In [9]:
# file_full_name = '../../label/political_news_label_0_to_99.csv'
file_full_name = "./v1_Total_Opinion.csv"
file_output_name = file_full_name.split(".csv")[0]

news_df = pd.read_csv(file_full_name)
# all_output_file = "./v0" + file_output_name + "_result.csv"
# opinion_after_cluster_file = "./v0" + file_output_name + "_all_opinions_after_cluster.csv"
all_output_file = "v1_Total_Opinion.csv"
opinion_after_cluster_file = "v1_cluster_opinions.csv"
topic_info_file = "v1_topic_info.csv"

In [10]:
# news_df = pd.read_csv('./v1_Total_Opinion.csv')

In [11]:
news_df.head(1)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,annotation_id,created_at,updated_at,lead_time,article_coref,people_name,ner,one_to_one_pair,one_to_one_pair_count
0,883,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...",政治,3,陶本和,1634110500,蔡英文發聲挺蘇貞昌 批鄭麗文情緒性發言對當事人不公,2100273,"[{""start"": 0, ""end"": 142, ""text"": ""針對行政院長蘇貞昌、國...",1,37,2022-02-03T14:55:55.310663Z,2022-02-03T15:02:18.598120Z,518.272,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...","['蔡英文', '行政院長', '鄭麗文', '民進黨', '蘇貞昌', '台灣人民', '...","['蔡英文', '行政院長', '12日', '鄭麗文', '台灣', '民進黨', '蘇貞...","[[{'start': 134, 'end': 138, 'text': '台灣人民', '...",9


#### 輸入 ckip 帳號密碼

In [12]:
pipeline = CkipPipeline(opts={'con_parser': {'username': '', 'password': ''}})

## 4. 找出結尾與開頭
原則為使用 one_to_one pair 到"。"或"；"為止，再根據以下條件過濾  
1. VE2 一定要在 agent 後面
2. 主詞為「你我他」就放棄
3. 意見小於10字即放棄
4. 句首往後找"，" 如果找到，就當作是開頭
5. 句尾往後找"」" 如果找到，就當作是結尾

### function 定義

In [13]:
# from random import betavariate
def get_opinion(article,sentence_list,people_name_list):
  # print(type(people_name_list))
  if type(people_name_list) == str:
    people_name_list = list(ast.literal_eval(people_name_list))
  people_opinion_list = []

  tolerance = 3
  end_list = [m.start() for m in re.finditer('。|；|;', article)]
  end_list.append(0)
  end_list.append(len(article))
  end_list.sort()

  not_agent_list = ["你","妳","我","他","她"]
  if type(sentence_list) == str:
    sentence_list = ast.literal_eval(sentence_list)
  # sentence agent not he or she, and verb is not in front of agent
  sentence_list = [sentence for sentence in sentence_list if ((sentence[0]['text'] not in not_agent_list) and (sentence[0]['labels'] != ['VE2']))]

  order = 0
  for sentence in sentence_list:
    for i in range(len(end_list)-1):

      if end_list[i] - tolerance < sentence[0]['start'] and sentence[1]['end'] <= end_list[i+1]:
        # to find "," place
        start_point = sentence[1]['end']
        start_icon_list = [',']
        for temp_start in range(3):
          if article[start_point+temp_start] in start_icon_list:
            start_point = start_point + temp_start + 1
            break
        
        # to find "」" place
        end_point = end_list[i+1]
        end_icon_list = ["」"]
        for temp_end in range(3):
          if article[start_point + temp_end] in end_icon_list:
            start_point = start_point + temp_end + 1
            break
        if end_point - start_point < 10:
          continue
        opinion = article[start_point:end_point]

        # change the name by candidate name list
        people_name = sentence[0]['text']
        for people_name_candidate in people_name_list:
          if people_name_candidate in sentence[0]['text']:
            people_name = people_name_candidate
            break
        # check agent is in people name_list
        if people_name not in people_name_list:
          continue

        # people_opinion_object = {"person":people_name,"verb":sentence[1]['text'],"opinion":opinion,"order":order}
        people_opinion_object = [[{'start': sentence[0]['start'], 
                                  'end': sentence[0]['end'], 
                                  'text':people_name,
                                  'labels': ['person']
                                  },{
                                  'start':start_point,
                                  'end': end_point, 
                                  'text': opinion,
                                  'labels': ['opinion']
                                  },{
                                  'start': sentence[1]['start'],
                                  'end': sentence[1]['end'], 
                                  'text': sentence[1]['text'],
                                  'labels': ['verb']
                                  }
                                 ]
                                ,order
                                ]
        people_opinion_list.append(people_opinion_object)
        order = order + 1
        
  return people_opinion_list

## 6. 開始實作

4.

In [14]:
news_df.iloc[0]['one_to_one_pair']

"[[{'start': 134, 'end': 138, 'text': '台灣人民', 'labels': ['agent']}, {'start': 139, 'end': 141, 'text': '樂見', 'labels': ['VE2']}], [{'start': 142, 'end': 145, 'text': '蔡英文', 'labels': ['agent']}, {'start': 150, 'end': 152, 'text': '表示', 'labels': ['VE2']}], [{'start': 258, 'end': 261, 'text': '蔡英文', 'labels': ['agent']}, {'start': 261, 'end': 262, 'text': '說', 'labels': ['VE2']}], [{'start': 269, 'end': 270, 'text': '她', 'labels': ['agent']}, {'start': 270, 'end': 272, 'text': '看到', 'labels': ['VE2']}], [{'start': 339, 'end': 340, 'text': '她', 'labels': ['agent']}, {'start': 340, 'end': 342, 'text': '認為', 'labels': ['VE2']}], [{'start': 414, 'end': 417, 'text': '蔡英文', 'labels': ['agent']}, {'start': 417, 'end': 419, 'text': '重申', 'labels': ['VE2']}], [{'start': 509, 'end': 512, 'text': '蔡英文', 'labels': ['agent']}, {'start': 514, 'end': 515, 'text': '說', 'labels': ['VE2']}], [{'start': 630, 'end': 633, 'text': '蔡英文', 'labels': ['agent']}, {'start': 633, 'end': 635, 'text': '強調', 'labels'

In [15]:
news_df['opinions_list'] = ""

for i in trange(len(news_df)):
    if i == 5:
        break
    try:
        news_df['opinions_list'][i] = get_opinion(news_df.iloc[i]['article_coref'],news_df.iloc[i]['one_to_one_pair'],news_df.iloc[i]['people_name'])
        # print(news_df['opinions_no_he_and_she'][i])
    except:
        print('error in ',i)
        news_df['opinions_list'][i] = []

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_13024/1444996564.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['opinions_list'][i] = get_opinion(news_df.iloc[i]['article_coref'],news_df.iloc[i]['one_to_one_pair'],news_df.iloc[i]['people_name'])
  5%|▌         | 5/100 [00:00<00:00, 111.11it/s]


In [16]:
news_df.head(5)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,annotation_id,created_at,updated_at,lead_time,article_coref,people_name,ner,one_to_one_pair,one_to_one_pair_count,opinions_list
0,883,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...",政治,3,陶本和,1634110500,蔡英文發聲挺蘇貞昌 批鄭麗文情緒性發言對當事人不公,2100273,"[{""start"": 0, ""end"": 142, ""text"": ""針對行政院長蘇貞昌、國...",1,37,2022-02-03T14:55:55.310663Z,2022-02-03T15:02:18.598120Z,518.272,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...","['蔡英文', '行政院長', '鄭麗文', '民進黨', '蘇貞昌', '台灣人民', '...","['蔡英文', '行政院長', '12日', '鄭麗文', '台灣', '民進黨', '蘇貞...","[[{'start': 134, 'end': 138, 'text': '台灣人民', '...",9,"[[[{'start': 142, 'end': 145, 'text': '蔡英文', '..."
1,862,"行政院長蘇貞昌12日在立法院總質詢時,怒斥國民黨立委鄭麗文「袂見笑」、「當時還要拜託我去幫你...",政治,3,呂晏慈,1634092560,蘇貞昌酸鄭麗文「民進黨叛徒」 他反譏:蔡英文也當國民黨閣員,2100056,"[{""start"": 0, ""end"": 51, ""text"": ""行政院長蘇貞昌12日在立...",1,16,2022-02-03T12:47:32.966394Z,2022-02-03T12:47:43.626597Z,30432.838,"行政院長蘇貞昌12日在立法院總質詢時,怒斥國民黨立委鄭麗文「袂見笑」、「當時還要拜託我去幫你...","['袂見', '蘇貞昌', '民進黨', '鄭麗文', '李德維', '陳明文', '國民黨...","['袂見', '蘇貞昌', '蘇', '費鴻泰酸', '蘇貞昌', '鄭先', '民進黨',...","[[{'start': 86, 'end': 94, 'text': '黨團總召費鴻泰酸',...",7,"[[[{'start': 86, 'end': 94, 'text': '費鴻泰', 'la..."
2,854,"台灣基進立委陳柏惟遭罷免團體掛看板質疑「反對採購疫苗」,立法院副院長蔡其昌4日反駁,陳柏惟反...",政治,3,呂晏慈,1634090220,蔡其昌質疑「想買中國科興疫苗」 國民黨團今按鈴控告妨害名譽,2100001,"[{""start"": 0, ""end"": 27, ""text"": ""台灣基進立委陳柏惟遭罷免...",1,8,2022-02-03T05:22:16.944255Z,2022-02-03T06:27:27.039925Z,7616.256,"台灣基進立委陳柏惟遭罷免團體掛看板質疑「反對採購疫苗」,立法院副院長蔡其昌4日反駁,陳柏惟反...","['國民黨團', 'BNT', '蔡其昌', '中國科興', '陳柏惟', '莫德納', '...","['上午', '國民黨團', '4日', '中國', 'BNT', '蔡其昌', '國民黨團...","[[{'start': 0, 'end': 9, 'text': '台灣基進立委陳柏惟', ...",3,"[[[{'start': 0, 'end': 9, 'text': '陳柏惟', 'labe..."
3,953,"實況主丁特日前因質疑「天堂M」遊戲道具製作機率公告不實,引起各界關注。民眾黨立委高虹安14日...",政治,3,張鈞量,1634184240,丁特控「天堂M」製作機率造假 高虹安揪朝野立委推「轉蛋法」修法,2101034,"[{""start"": 0, ""end"": 35, ""text"": ""實況主丁特日前因質疑「天...",1,107,2022-02-04T13:48:13.063893Z,2022-02-04T13:48:13.063922Z,263.836,"實況主丁特日前因質疑「天堂M」遊戲道具製作機率公告不實,引起各界關注。民眾黨立委高虹安14日...","['時代力量', '高虹安', '虹安', '紫布', 'Microsoft', '洪孟楷'...","['時代力量', '高虹安', '虹安', '2018年', '高虹安', '上百萬', '...","[[{'start': 15, 'end': 19, 'text': '遊戲道具', 'la...",9,"[[[{'start': 35, 'end': 43, 'text': '高虹安', 'la..."
4,952,"國防部長邱國正14日證實,長年從事後備動員業務規劃的國防部全民防衛動員綜合作業室主任韓岡明,...",政治,3,林銘翰,1634182920,全民防衛動員署明年成立 全動室主任韓岡明屆齡退休,2101023,"[{""start"": 0, ""end"": 106, ""text"": ""國防部長邱國正14日證...",1,106,2022-02-04T13:43:48.981022Z,2022-02-04T13:43:48.981050Z,174.457,"國防部長邱國正14日證實,長年從事後備動員業務規劃的國防部全民防衛動員綜合作業室主任韓岡明,...","['全民防衛動員署', '陳以信', '國防部', '邱國正', '韓岡明', '國民黨',...","['全民防衛動員署', '陳以信', '一半', '國防委員會', '國防部', '邱國正'...","[[{'start': 0, 'end': 7, 'text': '國防部長邱國正', 'l...",8,"[[[{'start': 0, 'end': 7, 'text': '國防部', 'labe..."


In [19]:
news_df.to_csv('./v1_Total_Opinion.csv', index=False)

In [18]:
# news_df.iloc[-1]['opinions_list']